<a href="https://colab.research.google.com/github/mdessolis/pythontps/blob/main/Threading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmazione Multithread #

## Differenza tra multithread e multiprocessing ##

Per **thread** si intende un blocco di istruzioni (di solito una funzione o un metodo) eseguito all'interno di un processo in parallelo con altre istruzioni.
  
Pensiamo ad esempio ad un browser che deve mostrare una pagina web: se questa contiene diverse immagini/fogli di stile o altri elementi esterni il browser dovrà scaricare tutti questi file. Ebbene, di solito questo lo fa in parallelo facendo partire una procedura di download (thread) per ciascun file in modo da ottimizzare i tempi.

Oppure pensiamo ad un qualunque sistema di videoscrittura: mentre digitiamo il testo questo automaticamente viene impaginato e spesso anche controllato come ortografia. Ebbene, queste operazioni sono probabilmente eseguite da thread paralleli alla digitazione.

Nel caso del thread quindi si lavora sempre all'interno di un singolo processo di esecuzione che ha al suo interno delle diramazioni di esecuzione che lavorano in parallelo. Però, essendo un singolo processo, l'area dati e i registri di esecuzione del codice sono comuni, quindi i thread condividono tutti gli stessi dati.

Per **processo** si intende invece un blocco di codice che viene eseguito in genere in modo autonomo sul computer e al quale viene quindi assegnato un proprio spazio di memoria. Ad esempio se avviamo un programma di videoscrittura verrà creato un processo di esecuzione e assegnato a questo un codice univoco (detto PID). Nulla ci impedisce però di lanciare due esecuzioni contemporanee del programma, creando però due processi che non condividono tra loro nessuno spazio di memoria.

E' però possibile programmare un codice per istruirlo a dividere l'esecuzione del processo in più sottoprocessi da avviare in parallelo, sfruttando in questo modo le caratteristiche dei moderni processori, composti da più core che possono tranquillamente eseguire operazion in parallelismo reale.

A differenza però di una esecuzione multithread, in quella multiprocess non vi è condivisione della memoria per cui bisogna prevedere, nel caso se ne abbia bisogno, strumenti per far comunicare tra loro i sottoprocessi.

Vediamo ora il tutto con degli esempi.

## Problema: conta numeri primi ##

Si voglia contare quanti numeri primi sono presenti in un certo intervallo fornito in input.

Innanzitutto dovremo definire una funzione per stabilire se un numero è primo o no:


In [ ]:
def is_prime(x):
    """ restituisce True se x è primo, False altrimenti """
    if x<2: return False
    i = 2
    while i*i <= x:
        if x % i == 0: return False
        i += 1
    return True

# Esempio di uso:
print(f"5 is prime? {is_prime(5)}")
print(f"6 is prime? {is_prime(6)}")

5 is prime? True
6 is prime? False


Successivamente definiamo una funzione per contare quanti primi troviamo in un dato intervallo:

In [ ]:
def count_primes(a,b):
    """ Conta i numeri primi trovati all'interno dell'intervallo [a,b[ """
    count = 0
    for i in range(a, b):
        if is_prime(i):
            count +=1
    print(f"In [{a}, {b}] ho trovato {count} numeri primi")
    return count

# Esempio di uso

count_primes(1,10000)


In [1, 10000] ho trovato 1229 numeri primi


1229

La funzione count_primes si può scrivere in tanti modi differenti in Python. Sfruttando alcune funzioni e strutture del linguaggio potremmo per esempio scriverla anche così:

In [ ]:
def count_primesV2(a,b):
    """ Conta i numeri primi trovati all'interno dell'intervallo [a,b[ """
    """ memorizza nella variabile count la somma di una serie di 1 corrispondenti ad ogni primo che trova nell'intervallo """
    count = sum(1 for i in range(a, b) if is_prime(i))
    print(f"In [{a}, {b}] ho trovato {count} numeri primi")
    return count

# Esempio di uso

count_primesV2(1,10000)


In [1, 10000] ho trovato 1229 numeri primi


1229

Proviamo adesso a calcolare il tempo esatto di esecuzione della funzione cont_primes per un intervallo abbastanza grande. Per far questo richiamiamo la libreria **time** che ci mette a disposizione la funzione time() che restituisce un float con il tempo attuale in secondi calcolato a partire dall'1/1/70 (Epoch). Volendo ci sono anche funzioni differenti per il calcolo del tempo ma questa è generica e la possiamo applicare bene a tanti contesti.

In [ ]:
from time import time

start = time() # memorizziamo in start l'ora corrente
count_primes(1, 1_000_000) # eseguiamo la funzione
tempo = time() - start # calcoliamo in tempo il tempo trascorso

print(f"Tempo impiegato: {tempo:.2f} secondi")

# Facciamo la stessa prova con count_primesV2 e vediamo se cambiano i tempi

start = time() # memorizziamo in start l'ora corrente
count_primesV2(1, 1_000_000) # eseguiamo la funzione
tempo = time() - start # calcoliamo in tempo il tempo trascorso

print(f"Tempo impiegato: {tempo:.2f} secondi")

In [1, 1000000] ho trovato 78498 numeri primi
Tempo impiegato: 20.38 secondi
In [1, 1000000] ho trovato 78498 numeri primi
Tempo impiegato: 12.47 secondi


Bene, quello che abbiamo appena fatto è un algoritmo sviluppato con la programmazione tradizionale impostata pensando di far svolgere i calcoli ad un solo processore.

Però potremmo chiederci: ma anziché fare un unico ciclo in count_primes per tutto l'intervallo, non potremmo dividere il problema su più intervalli da far eseguire il più possibile in parallelo?

Qui intervengono i **thread**.

Andiamo però per passi: prima dividiamo l'unico intervallo in n intervalli che eseguiamo uno di seguito all'altro:


In [ ]:
inizio = 1
fine   = 1_000_000
n      = 5 # numero di intervalli
step   = (fine-inizio+1) // n # ampiezza di ogni intervallo

start = time()

conta = 0
for i in range(n):
    conta += count_primes(inizio + i*step, inizio + (i+1) * step)

print(f"In [{inizio}, {fine}] ho trovato {conta} numeri primi")

tempo = time() - start # calcoliamo in tempo il tempo trascorso

print(f"Tempo impiegato: {tempo:.2f} secondi")

In [1, 200001] ho trovato 17984 numeri primi
In [200001, 400001] ho trovato 15876 numeri primi
In [400001, 600001] ho trovato 15238 numeri primi
In [600001, 800001] ho trovato 14853 numeri primi
In [800001, 1000001] ho trovato 14547 numeri primi
In [1, 1000000] ho trovato 78498 numeri primi
Tempo impiegato: 10.65 secondi


Bene, ora dobbiamo istruire Python ad eseguire le funzioni count_primes in parallelo mediante thread. 

Per farlo Python mette a disposizione il modulo **threading** con la classe **Thread** che ci consente proprio di avviare una funzione in un nuovo thread.
 

In [ ]:
from threading import Thread

inizio = 1
fine   = 1_000_000
n      = 5 # numero di intervalli
step   = (fine-inizio+1) // n # ampiezza di ogni intervallo

start = time()

# creiamo un array in cui memorizzare i vari thread
t = []

for i in range(n):

    # creiamo un instanza della classe Thread e associamo la funzione da eseguire
    thread = Thread(target = count_primes, args=(inizio + i*step, inizio + (i+1) * step))
    
    # facciamo partire il thread
    thread.start()
    
    # aggiungiamo il thread all'array
    t.append(thread)

# comunichiamo al programma di attendere la fine di tutti i thread prima di proseguire
for thread in t: thread.join()

tempo = time() - start # calcoliamo in tempo il tempo trascorso

print(f"Tempo impiegato: {tempo:.2f} secondi")

In [1, 200001] ho trovato 17984 numeri primi
In [200001, 400001] ho trovato 15876 numeri primi
In [600001, 800001] ho trovato 14853 numeri primi
In [400001, 600001] ho trovato 15238 numeri primi
In [800001, 1000001] ho trovato 14547 numeri primi
Tempo impiegato: 10.22 secondi


Notare che in queste ultime istruzioni è stato rimosso il contatore complessivo dei numeri trovati. Questo perché quando si eseguono le funzioni come thread non si può ottenere il valore restituito dalla funzione stessa, quindi occorrerà modificare il codice della funzione in modo che possa modificare dal suo interno il contatore.

Dall'esecuzione su questo notebook si può notare che non ci sono particolari vantaggi nei tempi. Provando però ad eseguirlo, anche più volte, sul proprio pc si può notare un piccolo miglioramento. 

A dire il vero in Python i thread non sono gestiti come in C++, dove il tempo di esecuzione verrebbe apprezzato di più. 
Di fatto in Python i thread, a causa di come è implementato l'interprete dei comandi, sono eseguiti uno alla volta e il parallelismo è solo simulato. 

Apprezzeremo di più l'esecuzione parallela se impostiamo il programma in versione non multi-thread ma multi-process.

Provare sul proprio pc la seguente versione:

In [ ]:
from multiprocessing import Process

if __name__ == "__main__":
    """ Questa if serve per far eseguire le istruzioni seguenti solo al processo principale """
    inizio = 1
    fine   = 1_000_000
    n      = 5 # numero di intervalli
    step   = (fine-inizio+1) // n # ampiezza di ogni intervallo

    start = time()

    # creiamo un array in cui memorizzare i vari thread
    t = []

    for i in range(n):

        # creiamo un instanza della classe Process e associamo la funzione da eseguire
        p = Process(target = count_primes, args=(inizio + i*step, inizio + (i+1) * step))
        
        # facciamo partire il processo
        p.start()
        
        # aggiungiamo il processo all'array
        t.append(p)

    # comunichiamo al programma di attendere la fine di tutti i processi prima di proseguire
    for p in t: p.join()

    tempo = time() - start # calcoliamo in tempo il tempo trascorso

    print(f"Tempo impiegato: {tempo:.2f} secondi")

In [1, 200001] ho trovato 17984 numeri primi
In [200001, 400001] ho trovato 15876 numeri primi
In [400001, 600001] ho trovato 15238 numeri primi
In [600001, 800001] ho trovato 14853 numeri primi
In [800001, 1000001] ho trovato 14547 numeri primi
Tempo impiegato: 11.88 secondi
